In [16]:
import cv2
import numpy as np
import mediapipe as mp

In [17]:
import os, glob

glob.glob("*.prototxt"), glob.glob("*.caffemodel")


(['pose_deploy_linevec.prototxt'], ['pose_iter_440000.caffemodel'])

In [23]:
PROTO = "pose_deploy_linevec.prototxt"
WEIGHTS = "pose_iter_440000.caffemodel"

net = cv2.dnn.readNetFromCaffe(PROTO, WEIGHTS)


In [21]:
COCO = {
    "R_HIP": 8,
    "R_KNEE": 9,
    "R_ANKLE": 10,
    "L_HIP": 11,
    "L_KNEE": 12,
    "L_ANKLE": 13,
}


In [24]:
def angle_deg(a, b, c):
    ba = a - b
    bc = c - b
    nba = np.linalg.norm(ba)
    nbc = np.linalg.norm(bc)
    if nba < 1e-6 or nbc < 1e-6:
        return np.nan
    cosang = np.dot(ba, bc) / (nba * nbc)
    cosang = np.clip(cosang, -1.0, 1.0)
    return float(np.degrees(np.arccos(cosang)))

def infer_keypoints(frame, thresh=0.15, in_size=(368, 368)):
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, in_size, swapRB=False, crop=False)
    net.setInput(blob)
    out = net.forward()

    pts = {}
    for name, idx in COCO.items():
        hm = out[0, idx, :, :]
        _, conf, _, pt = cv2.minMaxLoc(hm)
        if conf > thresh:
            x = int(w * pt[0] / hm.shape[1])
            y = int(h * pt[1] / hm.shape[0])
            pts[name] = np.array([x, y], dtype=float)
        else:
            pts[name] = None
    return pts

def knee_angles(pts):
    L = (pts["L_HIP"], pts["L_KNEE"], pts["L_ANKLE"])
    R = (pts["R_HIP"], pts["R_KNEE"], pts["R_ANKLE"])
    return {
        "L_knee": angle_deg(*L) if None not in L else np.nan,
        "R_knee": angle_deg(*R) if None not in R else np.nan,
    }

def ema(prev, x, alpha=0.25):
    if prev is None or (isinstance(prev, float) and np.isnan(prev)):
        return x
    if isinstance(x, float) and np.isnan(x):
        return prev
    return (1 - alpha) * prev + alpha * x


In [ ]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Camera not available")

sL = None
sR = None

try:
    while True:
        ok, frame = cap.read()
        if not ok:
            break

        frame = cv2.flip(frame, 1)
        pts = infer_keypoints(frame)
        ang = knee_angles(pts)

        sL = ema(sL, ang["L_knee"])
        sR = ema(sR, ang["R_knee"])

        for v in pts.values():
            if v is not None:
                cv2.circle(frame, tuple(v.astype(int)), 4, (0, 255, 0), -1)

        cv2.putText(frame, f"L knee: {sL:.1f} deg", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        cv2.putText(frame, f"R knee: {sR:.1f} deg", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        cv2.putText(frame, "Press q to quit", (10, 95),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

        cv2.imshow("Walking: knee angles (OpenPose COCO)", frame)
        if (cv2.waitKey(1) & 0xFF) == ord("q"):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()
